## roborobo라는 자율주행 로봇을 만들고, 그 로봇을 자율적으로 주행하게 하는 코드를 짜 보자.

In [5]:
#robokits에 대한 라이브러리를 설치한다.

!pip install RobokitRS

   ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
   ---------------------------------------- 90.6/90.6 kB 2.6 MB/s eta 0:00:00


모듈 불러오기/색상 설정 등

In [3]:
#모듈 불러오기
#모듈에 로봇과 연결하고, 로봇을 움직이는 함수들을 정의했다.
#import mobility_module as m #이 코드를 실행하면 모듈 파일에 있는 코드가 실행되어야 한다.
import time as t #시간에 대한 함수 정의.
#이런 방법으로도 모듈의 함수를 간단히 불러올 수 있다.
from mobility_module import *


C:\Users\User\anaconda3\Lib\site-packages\RobokitRS\pins
RSBoard serial port opend
Thread init done
Pin mode init done
Sysex init done
RSBoard init done
연결 완료!


In [3]:
from RobokitRS.RobokitRS import *
import time as t


rs = RobokitRS() #객체 담기

C:\Users\User\anaconda3\Lib\site-packages\RobokitRS\pins


In [5]:
#모듈의 함수를 사용해보자.
#함수를 따로 저장하니 코드에 드는 힘이 절감되는 장점이 있다.

forward()
t.sleep(1)
stop()

forward()
t.sleep(1)
stop()

In [16]:
#모터가 돌아가는지 아닌지 확인한다. 
#moter_write함수를 통해 모터를 돌아가게 한다.

#motor_write의 매개변수들을 바꿔보면서 기계를 동작해보자.
rs.motor_write(0,1,10) #첫번째는 모터,두번째는 방향,세번째는 속도이다.
t.sleep(1)
rs.motor_write(2,1,10)
stop()


### 여러 방향으로 이동하는 함수 만들기

앞에서 배웠던 motor_write 함수들을 이용해서, 좌/우 상/하향,우측,좌측 코드를 만든다.

스피드를 매개변수로 받는것이 필요하다.

In [7]:
#좌하향
def left_down(i=10): #기본값은 10
    rs.motor_write(1,1,i)
    rs.motor_write(3,1,i)
#우상향
def right_up(i=10):
    rs.motor_write(0,1,i)
    rs.motor_write(2,1,i)
#전진
def reverse(i=10):
    rs.motor_write(0,0,i)
    rs.motor_write(1,1,i)
    rs.motor_write(2,1,i)
    rs.motor_write(3,0,i)
#정지
def stop():
    rs.motor_write(0,1,0)
    rs.motor_write(1,1,0)
    rs.motor_write(2,1,0)
    rs.motor_write(3,1,0)
#후진
def forward(i=10):
    rs.motor_write(0,1,i)
    rs.motor_write(1,0,i)
    rs.motor_write(2,0,i)
    rs.motor_write(3,1,i)


#좌회전
def left(i=10):
    rs.motor_write(0,0,i)
    rs.motor_write(1,0,i)
    rs.motor_write(2,1,i)
    rs.motor_write(3,1,i)

#우회전
def right(i=10):
    rs.motor_write(0,1,i)
    rs.motor_write(1,1,i)
    rs.motor_write(2,0,i)
    rs.motor_write(3,0,i)
    

In [23]:
#실제로 한번 해보자. 1초동안 오른쪽으로 이동 한 뒤 멈추는 것을 알 수 있다.
left(15)
t.sleep(1)
stop()

색상 설정

In [11]:
#색상 설정

#13은 rgb선을 꼽은 곳의 번호이다.
rs.set_rgb_led_color(13,100,100,100) #우리가 잘 아는 rgb가 맞다.
rs.set_rgb_led_on(13)
t.sleep(3) #n초 간격으로 rgb_led_color에 맞는 색을 켠다.

#2.rgb_led끄기

rs.set_rgb_led_off(13)

초음파 센서 확인하기


In [9]:
#초음파 센서 관련함수
#초음파는 sonar로 표현한다.

#1.초음파센서 포트 연결하기

m.rs.sonar_begin(3)

#2.초음파센서 값 읽기

i = 0
while i<10:
    SONAR = m.rs.sonar_read(3)
    print(SONAR)
    t.sleep(1) #1초동안...
    i = i+1

#값이 0인 경우에는 아예 측정이 안되는 경우이므로, 처음부터 코드를 다시 실행!



#재미있는 실험을 하나 해볼까? 차량 앞에 무언가를 두면 그 거리에 따라 값이 줄어들거나 늘어나는 것을 알 수 있다.
#이것이 바로 sonar_read을 통해 초음파를 탐지하는 것의 증거이다.
#값을 측정할 수 없거나, 오류가 날 때는 508이 나온다. 이런 경우를 대비하기 위해 코드로 처리를 해야 한다.


0
508
144
104
140
145
144
144
141
104


### 초음파 센서를 활용해서 거리 탐지할 때 오류가 나는 경우를 핸들링하기

거리를 탐지할 때 오류가 나는 경우(508이 뜨는 경우)를 핸들링하자.

In [ ]:
while True:
    sonar = rs.sonar_read(3)#초음파 센서와 해당 포트가 연결되어 있다면 정의가 된다.
    print(sonar)
    t.sleep(0.1) #아주 약간의 간격을 둔다.

    if sonar==0 or sonar==508 or sonar==None:
        continue
    if sonar !=0 or sonar != 308:
        if sonar < 50:
            rs.set_rgb_led_color(13,100,0,0)
            stop()
            break
    else: #값이 508이나 0일 때 동작을 설정한다.
        stop()
        

### 초음파센서를 활용해서 장애물을 인식해서 속도를 조절하기

벽이 있다면, 벽 앞에서 멈출 수 있게 한다.


In [10]:
#우선 초음파를 이용해서 장애물을 감지하는 코드를 짠다. 

def obstacle():
    #1.초음파센서 포트 연결하기
    rs.sonar_begin(3)
    #2.while문을 이용해서 장애물 탐지
    while 1:
        obstacle = rs.sonar_read(3)
        t.sleep(1) #1초동안...

        print(obstacle)
        if(obstacle<30):
          
            stop() #장애물과 거리가 좁혀지면 멈춘다.
            break
        

In [16]:
#앞서 정의한 코드를 바탕으로 장애물을 탐지해서 멈출 수 있도록 한다. 

forward() #앞으로 가는 코드
obstacle() #장애물을 인식하면 멈추는 코드

96
86
72
60
47
35
23


In [ ]:
stop()


### 적외선 센서값 확인

적외선 센서를 통해 바닥의 색을 확인한다.

이는 자율주행에서 도로의 색을 인지하는데 활용된다.

In [10]:
#적외선 센서값 확인
#밑바닥의 색에 따라 출력값이 달라지는 것을 볼 수 있을 것이다.


for i in range(10):
    L = m.rs.analog_read(3)
    R = m.rs.analog_read(4)
    print(L,R)
    t.sleep(0.1)

815 814
815 814
815 814
815 814
815 814
815 814
815 814
815 814
815 814
815 814


### 낭떠러지가 발견이 되면 멈추도록 코드를 짜기

평범한 길을 벗어나서 낭떠러지가 나오면 멈추도록 한다.

이 때, 차의 속도를 너무 빠르게 하면 죽어버릴 수가 있으니 차의 속도를 적당히 느리게 하자.

In [22]:
t.sleep(1)

while True:
    R = m.rs.analog_read(3)
    t.sleep(0.1)
    print(R)

    if R> 100:
        forward(5)
    else:
        stop()
        break

print("테스트 종료.")

815
815
815
818
815
815
815
815
815
815
815
815
815
815
815
815
29
테스트 종료.


In [ ]:
#라인트레이서코드
 while True:

        L = rs.analog_read(1)

        R = rs.analog_read(0)

        print(L, R)

        t.sleep(0.2)

        

        if R > 100 :

        

            if L > 100 :

                forward(4)

            else :

                while L < 100 :

                    L_turn(4)

                    L = rs.analog_read(1)

                    print(L)

                    t.sleep(0.2)

            

        else: 

            

            if L > 100 :

                while R < 100:

                    R_turn(4)

                    R = rs.analog_read(0)

                    t.sleep(0.2)

            else :

                stop()

                t.sleep(1)

### 자율주행차량의 방향을 기울기를 통해 측정하는 기능 구현 - 자이로 센서




In [9]:
#자율주행차량의 방향을 기울기를 통해 측정하는 기능!(이를 위해서는 )
#회전을 하게 되면 값이 바뀌게 된다.
#gyro_begin을 통해 구동을 시작한다.
rs.gyro_begin()
rs.set_gyro_default_position()



In [11]:
left_down()
t.sleep(1)
stop()

In [13]:
#x,y,z의 방향을 실시간으로 측정한다. 차를 직접 이동시켜 보면 값이 나올걸? 
#rs가 충돌하지 않도록 관련 내용은 한번만 import를 한다. 

i=0

while 10>i:
    x = rs.gyro_read(GyroDataType.ANGLE_X)
    y = rs.gyro_read(GyroDataType.ANGLE_Y)
    z = rs.gyro_read(GyroDataType.ANGLE_Z)
    print("x=",x,"y=",y,"z=",z)
    t.sleep(1)
    i+=1

x= 0 y= -2 z= 40
x= 1 y= 0 z= 40
x= 0 y= 0 z= 40
x= 1 y= 0 z= 40
x= 0 y= 0 z= 40
x= 1 y= 0 z= 40
x= 0 y= 0 z= 40
x= 0 y= 0 z= 40
x= 1 y= 0 z= 40
x= 0 y= 0 z= 40


### 자이로센서 코드

In [ ]:
#위의 코드들을 응용해서 좌표에 따라 자동차가 동작하는 로직을 코딩해보자.

#자이로센서 코드-z축의 값에 따라 차량을 자율적으로 움직인다.
while True :

    z = rs.gyro_read(GyroDataType.ANGLE_Z) 

    print(z)

    t.sleep(1)

    

    if -5 <= z and z <= 5:

        forward()

        t.sleep(0.5)

        

    elif z < -5 :

        stop()

        t.sleep(0.1)

        #조건반복문

        while z < 0 :

            z = rs.gyro_read(GyroDataType.ANGLE_Z)

            t.sleep(0.05)

            right_up(2)

            print(2)

            

        stop()

        print("왼쪽완료")

    elif z > 5: 

        while z > 2 :

            z = rs.gyro_read(GyroDataType.ANGLE_Z)

            t.sleep(0.05)

            left_down(2)

            print(2)

            

        stop()

        print("오른쪽완료")